In [1]:
from pymongo import MongoClient
import pymongo
import pprint
import re

runtime_pat = re.compile(r'([0-9]+) min')

# using local connection to speed up
client = MongoClient("mongodb://localhost:27017")
db = client['analytics']
for movie in client.analytics.mflix.find({}).limit(100):
    fields_to_set = {}
    fields_to_unset = {}
    for k,v in movie.copy().items():
        if v=="" or v==[""]:
#           as we are deleting items from the dict we must use the a copy to iterate
            del movie[k]
#             fields_to_unset[k] = ""
        
#       here we use the non empty fields
        if 'director' in movie:
#             mark the field to deletion as we will create a new one with different name
            fields_to_unset['director'] = ""
            fields_to_set['directors'] = movie['director'].split(',')
        
        imdb = {}
        if 'imdbID' in movie:
            fields_to_unset['imdbID'] = ""
            imdb['id'] = movie['imdbID']
        if 'imdbRating' in movie:
            fields_to_unset['imdbRating'] = ""
            imdb['rating'] = movie['imdbRating']
        fields_to_set['imdb'] = imdb
            
#       here is something that we cant yet do in the aggregation framework, change type..
        if 'runtime' in movie:
            m = runtime_pat.match(movie['runtime'])
            if m:
                fields_to_set['runtime'] = int(m.group(1))
                
            
            
        updated_doc = {}
        if fields_to_set:
            updated_doc['$set'] = fields_to_set
        if fields_to_unset:
            updated_doc['$unset'] = fields_to_unset

        

        db.movies_updated.update_one({'_id': movie['_id']}, updated_doc, upsert=True)

All of these update documents will be composed of two keys, a \$set key and a \$unset key.
In the MongoDB query language, for update operations,\$set has the semantics of specifying a field that you either want to add to the document in question or a field that you want to update with a new value.
